In [15]:
#Import the libraries
import pandas as pd
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from tabulate import tabulate
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string
import numpy as np

In [16]:
#Load the data
df = pd.read_csv('customer_complaints_1.csv')

In [17]:
#Preprocessing - remove punctuation
string.punctuation

def remove_punctuation(text):
    punctuation_free = ""
    
    for i in text:
        if i not in string.punctuation:
            punctuation_free += i
            
    return punctuation_free

df['Clean_Punctuation'] = df['Text'].apply(remove_punctuation)

df[["Author", "Text", "Clean_Punctuation"]]

,Author,Text,Clean_Punctuation
0,"Alantae of Chesterfeild, MI",I used to love Comcast. Until all these consta...,I used to love Comcast Until all these constan...
1,"Vera of Philadelphia, PA",I'm so over Comcast! The worst internet provid...,Im so over Comcast The worst internet provider...
2,"Sarah of Rancho Cordova, CA",If I could give them a negative star or no sta...,If I could give them a negative star or no sta...
3,"Dennis of Manchester, NH",I've had the worst experiences so far since in...,Ive had the worst experiences so far since ins...
4,"Ryan of Bellevue, WA",Check your contract when you sign up for Comca...,Check your contract when you sign up for Comca...
5,"Terri of Mobile, AL",Thank God. I am changing to Dish. They gave me...,Thank God I am changing to Dish They gave me a...
6,"Kellie of Salt Lake City, UT",I Have been a long time customer and only have...,I Have been a long time customer and only have...
7,"Kathleen of New Haven, CT",There is a malfunction on the DVR manager whic...,There is a malfunction on the DVR manager whic...
8,"Shira of Bloomfield, NJ",Charges overwhelming. Comcast service rep was ...,Charges overwhelming Comcast service rep was s...
9,"Kristy of Alpharetta, GA","I have had cable, DISH, and U-verse, etc. in t...",I have had cable DISH and Uverse etc in the pa...


In [18]:
#Preprocessing - apply lowercase to all text
df['Clean_Lower'] = df['Clean_Punctuation'].str.lower()

df[["Author", "Text", "Clean_Punctuation", "Clean_Lower"]]

,Author,Text,Clean_Punctuation,Clean_Lower
0,"Alantae of Chesterfeild, MI",I used to love Comcast. Until all these consta...,I used to love Comcast Until all these constan...,i used to love comcast until all these constan...
1,"Vera of Philadelphia, PA",I'm so over Comcast! The worst internet provid...,Im so over Comcast The worst internet provider...,im so over comcast the worst internet provider...
2,"Sarah of Rancho Cordova, CA",If I could give them a negative star or no sta...,If I could give them a negative star or no sta...,if i could give them a negative star or no sta...
3,"Dennis of Manchester, NH",I've had the worst experiences so far since in...,Ive had the worst experiences so far since ins...,ive had the worst experiences so far since ins...
4,"Ryan of Bellevue, WA",Check your contract when you sign up for Comca...,Check your contract when you sign up for Comca...,check your contract when you sign up for comca...
5,"Terri of Mobile, AL",Thank God. I am changing to Dish. They gave me...,Thank God I am changing to Dish They gave me a...,thank god i am changing to dish they gave me a...
6,"Kellie of Salt Lake City, UT",I Have been a long time customer and only have...,I Have been a long time customer and only have...,i have been a long time customer and only have...
7,"Kathleen of New Haven, CT",There is a malfunction on the DVR manager whic...,There is a malfunction on the DVR manager whic...,there is a malfunction on the dvr manager whic...
8,"Shira of Bloomfield, NJ",Charges overwhelming. Comcast service rep was ...,Charges overwhelming Comcast service rep was s...,charges overwhelming comcast service rep was s...
9,"Kristy of Alpharetta, GA","I have had cable, DISH, and U-verse, etc. in t...",I have had cable DISH and Uverse etc in the pa...,i have had cable dish and uverse etc in the pa...


In [19]:
#Preprocessing - remove integer and (-)
def remove_numbers(text):
        return re.sub("[\d-]",'',text)
    
df['Clean_Number'] = df['Clean_Lower'].apply(remove_numbers)

df[["Author", "Text", "Clean_Punctuation", "Clean_Lower", "Clean_Number"]]

,Author,Text,Clean_Punctuation,Clean_Lower,Clean_Number
0,"Alantae of Chesterfeild, MI",I used to love Comcast. Until all these consta...,I used to love Comcast Until all these constan...,i used to love comcast until all these constan...,i used to love comcast until all these constan...
1,"Vera of Philadelphia, PA",I'm so over Comcast! The worst internet provid...,Im so over Comcast The worst internet provider...,im so over comcast the worst internet provider...,im so over comcast the worst internet provider...
2,"Sarah of Rancho Cordova, CA",If I could give them a negative star or no sta...,If I could give them a negative star or no sta...,if i could give them a negative star or no sta...,if i could give them a negative star or no sta...
3,"Dennis of Manchester, NH",I've had the worst experiences so far since in...,Ive had the worst experiences so far since ins...,ive had the worst experiences so far since ins...,ive had the worst experiences so far since ins...
4,"Ryan of Bellevue, WA",Check your contract when you sign up for Comca...,Check your contract when you sign up for Comca...,check your contract when you sign up for comca...,check your contract when you sign up for comca...
5,"Terri of Mobile, AL",Thank God. I am changing to Dish. They gave me...,Thank God I am changing to Dish They gave me a...,thank god i am changing to dish they gave me a...,thank god i am changing to dish they gave me a...
6,"Kellie of Salt Lake City, UT",I Have been a long time customer and only have...,I Have been a long time customer and only have...,i have been a long time customer and only have...,i have been a long time customer and only have...
7,"Kathleen of New Haven, CT",There is a malfunction on the DVR manager whic...,There is a malfunction on the DVR manager whic...,there is a malfunction on the dvr manager whic...,there is a malfunction on the dvr manager whic...
8,"Shira of Bloomfield, NJ",Charges overwhelming. Comcast service rep was ...,Charges overwhelming Comcast service rep was s...,charges overwhelming comcast service rep was s...,charges overwhelming comcast service rep was s...
9,"Kristy of Alpharetta, GA","I have had cable, DISH, and U-verse, etc. in t...",I have had cable DISH and Uverse etc in the pa...,i have had cable dish and uverse etc in the pa...,i have had cable dish and uverse etc in the pa...


In [20]:
#Preprocessing - tokenize data
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

df['Tokenize_Data'] = df['Clean_Number'].apply(word_tokenize)

df[["Author", "Text", "Clean_Punctuation", "Clean_Lower", "Clean_Number", "Tokenize_Data"]]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cyber\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Author,Text,Clean_Punctuation,Clean_Lower,Clean_Number,Tokenize_Data
0,"Alantae of Chesterfeild, MI",I used to love Comcast. Until all these consta...,I used to love Comcast Until all these constan...,i used to love comcast until all these constan...,i used to love comcast until all these constan...,"[i, used, to, love, comcast, until, all, these..."
1,"Vera of Philadelphia, PA",I'm so over Comcast! The worst internet provid...,Im so over Comcast The worst internet provider...,im so over comcast the worst internet provider...,im so over comcast the worst internet provider...,"[im, so, over, comcast, the, worst, internet, ..."
2,"Sarah of Rancho Cordova, CA",If I could give them a negative star or no sta...,If I could give them a negative star or no sta...,if i could give them a negative star or no sta...,if i could give them a negative star or no sta...,"[if, i, could, give, them, a, negative, star, ..."
3,"Dennis of Manchester, NH",I've had the worst experiences so far since in...,Ive had the worst experiences so far since ins...,ive had the worst experiences so far since ins...,ive had the worst experiences so far since ins...,"[ive, had, the, worst, experiences, so, far, s..."
4,"Ryan of Bellevue, WA",Check your contract when you sign up for Comca...,Check your contract when you sign up for Comca...,check your contract when you sign up for comca...,check your contract when you sign up for comca...,"[check, your, contract, when, you, sign, up, f..."
5,"Terri of Mobile, AL",Thank God. I am changing to Dish. They gave me...,Thank God I am changing to Dish They gave me a...,thank god i am changing to dish they gave me a...,thank god i am changing to dish they gave me a...,"[thank, god, i, am, changing, to, dish, they, ..."
6,"Kellie of Salt Lake City, UT",I Have been a long time customer and only have...,I Have been a long time customer and only have...,i have been a long time customer and only have...,i have been a long time customer and only have...,"[i, have, been, a, long, time, customer, and, ..."
7,"Kathleen of New Haven, CT",There is a malfunction on the DVR manager whic...,There is a malfunction on the DVR manager whic...,there is a malfunction on the dvr manager whic...,there is a malfunction on the dvr manager whic...,"[there, is, a, malfunction, on, the, dvr, mana..."
8,"Shira of Bloomfield, NJ",Charges overwhelming. Comcast service rep was ...,Charges overwhelming Comcast service rep was s...,charges overwhelming comcast service rep was s...,charges overwhelming comcast service rep was s...,"[charges, overwhelming, comcast, service, rep,..."
9,"Kristy of Alpharetta, GA","I have had cable, DISH, and U-verse, etc. in t...",I have had cable DISH and Uverse etc in the pa...,i have had cable dish and uverse etc in the pa...,i have had cable dish and uverse etc in the pa...,"[i, have, had, cable, dish, and, uverse, etc, ..."


In [21]:
#Preprocessing - remove stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    output = []
    for i in text:
        if i not in stopwords:
            output.append(i)
    return output

df['Clean_Stopwords'] = df['Tokenize_Data'].apply(remove_stopwords)

df[["Author", "Text", "Clean_Punctuation", "Clean_Lower", "Clean_Number", "Tokenize_Data", "Clean_Stopwords"]]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cyber\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Author,Text,Clean_Punctuation,Clean_Lower,Clean_Number,Tokenize_Data,Clean_Stopwords
0,"Alantae of Chesterfeild, MI",I used to love Comcast. Until all these consta...,I used to love Comcast Until all these constan...,i used to love comcast until all these constan...,i used to love comcast until all these constan...,"[i, used, to, love, comcast, until, all, these...","[used, love, comcast, constant, updates, inter..."
1,"Vera of Philadelphia, PA",I'm so over Comcast! The worst internet provid...,Im so over Comcast The worst internet provider...,im so over comcast the worst internet provider...,im so over comcast the worst internet provider...,"[im, so, over, comcast, the, worst, internet, ...","[im, comcast, worst, internet, provider, im, t..."
2,"Sarah of Rancho Cordova, CA",If I could give them a negative star or no sta...,If I could give them a negative star or no sta...,if i could give them a negative star or no sta...,if i could give them a negative star or no sta...,"[if, i, could, give, them, a, negative, star, ...","[could, give, negative, star, stars, review, w..."
3,"Dennis of Manchester, NH",I've had the worst experiences so far since in...,Ive had the worst experiences so far since ins...,ive had the worst experiences so far since ins...,ive had the worst experiences so far since ins...,"[ive, had, the, worst, experiences, so, far, s...","[ive, worst, experiences, far, since, install,..."
4,"Ryan of Bellevue, WA",Check your contract when you sign up for Comca...,Check your contract when you sign up for Comca...,check your contract when you sign up for comca...,check your contract when you sign up for comca...,"[check, your, contract, when, you, sign, up, f...","[check, contract, sign, comcast, advertised, o..."
5,"Terri of Mobile, AL",Thank God. I am changing to Dish. They gave me...,Thank God I am changing to Dish They gave me a...,thank god i am changing to dish they gave me a...,thank god i am changing to dish they gave me a...,"[thank, god, i, am, changing, to, dish, they, ...","[thank, god, changing, dish, gave, awesome, pr..."
6,"Kellie of Salt Lake City, UT",I Have been a long time customer and only have...,I Have been a long time customer and only have...,i have been a long time customer and only have...,i have been a long time customer and only have...,"[i, have, been, a, long, time, customer, and, ...","[long, time, customer, xfinity, isp, local, wa..."
7,"Kathleen of New Haven, CT",There is a malfunction on the DVR manager whic...,There is a malfunction on the DVR manager whic...,there is a malfunction on the dvr manager whic...,there is a malfunction on the dvr manager whic...,"[there, is, a, malfunction, on, the, dvr, mana...","[malfunction, dvr, manager, preventing, us, ad..."
8,"Shira of Bloomfield, NJ",Charges overwhelming. Comcast service rep was ...,Charges overwhelming Comcast service rep was s...,charges overwhelming comcast service rep was s...,charges overwhelming comcast service rep was s...,"[charges, overwhelming, comcast, service, rep,...","[charges, overwhelming, comcast, service, rep,..."
9,"Kristy of Alpharetta, GA","I have had cable, DISH, and U-verse, etc. in t...",I have had cable DISH and Uverse etc in the pa...,i have had cable dish and uverse etc in the pa...,i have had cable dish and uverse etc in the pa...,"[i, have, had, cable, dish, and, uverse, etc, ...","[cable, dish, uverse, etc, past, eh, know, com..."


In [22]:
#Preprocessing - stemming
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

def stemming(text):
    stem_text = []
    for word in text:
        stemmed_word = porter_stemmer.stem(word)
        stem_text.append(stemmed_word)
    return stem_text

df['Clean_StemmedWord'] = df['Clean_Stopwords'].apply(stemming)

df[["Author", "Text", "Clean_Punctuation", "Clean_Lower", "Clean_Number", "Tokenize_Data", "Clean_Stopwords", "Clean_StemmedWord"]]

,Author,Text,Clean_Punctuation,Clean_Lower,Clean_Number,Tokenize_Data,Clean_Stopwords,Clean_StemmedWord
0,"Alantae of Chesterfeild, MI",I used to love Comcast. Until all these consta...,I used to love Comcast Until all these constan...,i used to love comcast until all these constan...,i used to love comcast until all these constan...,"[i, used, to, love, comcast, until, all, these...","[used, love, comcast, constant, updates, inter...","[use, love, comcast, constant, updat, internet..."
1,"Vera of Philadelphia, PA",I'm so over Comcast! The worst internet provid...,Im so over Comcast The worst internet provider...,im so over comcast the worst internet provider...,im so over comcast the worst internet provider...,"[im, so, over, comcast, the, worst, internet, ...","[im, comcast, worst, internet, provider, im, t...","[im, comcast, worst, internet, provid, im, tak..."
2,"Sarah of Rancho Cordova, CA",If I could give them a negative star or no sta...,If I could give them a negative star or no sta...,if i could give them a negative star or no sta...,if i could give them a negative star or no sta...,"[if, i, could, give, them, a, negative, star, ...","[could, give, negative, star, stars, review, w...","[could, give, neg, star, star, review, would, ..."
3,"Dennis of Manchester, NH",I've had the worst experiences so far since in...,Ive had the worst experiences so far since ins...,ive had the worst experiences so far since ins...,ive had the worst experiences so far since ins...,"[ive, had, the, worst, experiences, so, far, s...","[ive, worst, experiences, far, since, install,...","[ive, worst, experi, far, sinc, instal, noth, ..."
4,"Ryan of Bellevue, WA",Check your contract when you sign up for Comca...,Check your contract when you sign up for Comca...,check your contract when you sign up for comca...,check your contract when you sign up for comca...,"[check, your, contract, when, you, sign, up, f...","[check, contract, sign, comcast, advertised, o...","[check, contract, sign, comcast, advertis, off..."
5,"Terri of Mobile, AL",Thank God. I am changing to Dish. They gave me...,Thank God I am changing to Dish They gave me a...,thank god i am changing to dish they gave me a...,thank god i am changing to dish they gave me a...,"[thank, god, i, am, changing, to, dish, they, ...","[thank, god, changing, dish, gave, awesome, pr...","[thank, god, chang, dish, gave, awesom, price,..."
6,"Kellie of Salt Lake City, UT",I Have been a long time customer and only have...,I Have been a long time customer and only have...,i have been a long time customer and only have...,i have been a long time customer and only have...,"[i, have, been, a, long, time, customer, and, ...","[long, time, customer, xfinity, isp, local, wa...","[long, time, custom, xfiniti, isp, local, walm..."
7,"Kathleen of New Haven, CT",There is a malfunction on the DVR manager whic...,There is a malfunction on the DVR manager whic...,there is a malfunction on the dvr manager whic...,there is a malfunction on the dvr manager whic...,"[there, is, a, malfunction, on, the, dvr, mana...","[malfunction, dvr, manager, preventing, us, ad...","[malfunct, dvr, manag, prevent, us, ad, record..."
8,"Shira of Bloomfield, NJ",Charges overwhelming. Comcast service rep was ...,Charges overwhelming Comcast service rep was s...,charges overwhelming comcast service rep was s...,charges overwhelming comcast service rep was s...,"[charges, overwhelming, comcast, service, rep,...","[charges, overwhelming, comcast, service, rep,...","[charg, overwhelm, comcast, servic, rep, ignor..."
9,"Kristy of Alpharetta, GA","I have had cable, DISH, and U-verse, etc. in t...",I have had cable DISH and Uverse etc in the pa...,i have had cable dish and uverse etc in the pa...,i have had cable dish and uverse etc in the pa...,"[i, have, had, cable, dish, and, uverse, etc, ...","[cable, dish, uverse, etc, past, eh, know, com...","[cabl, dish, uvers, etc, past, eh, know, comca..."


In [23]:
#Preprocessing - lemmatizing

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatizer(text):
    lemm_text = []
    for word in text:
        lemmatized_word = wordnet_lemmatizer.lemmatize(word)
        lemm_text.append(lemmatized_word)
    return lemm_text

df['Clean_LemmatizedWord'] = df['Clean_StemmedWord'].apply(lemmatizer)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cyber\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
df[["Text", "Clean_LemmatizedWord"]]

,Text,Clean_LemmatizedWord
0,I used to love Comcast. Until all these consta...,"[use, love, comcast, constant, updat, internet..."
1,I'm so over Comcast! The worst internet provid...,"[im, comcast, worst, internet, provid, im, tak..."
2,If I could give them a negative star or no sta...,"[could, give, neg, star, star, review, would, ..."
3,I've had the worst experiences so far since in...,"[ive, worst, experi, far, sinc, instal, noth, ..."
4,Check your contract when you sign up for Comca...,"[check, contract, sign, comcast, advertis, off..."
5,Thank God. I am changing to Dish. They gave me...,"[thank, god, chang, dish, gave, awesom, price,..."
6,I Have been a long time customer and only have...,"[long, time, custom, xfiniti, isp, local, walm..."
7,There is a malfunction on the DVR manager whic...,"[malfunct, dvr, manag, prevent, u, ad, record,..."
8,Charges overwhelming. Comcast service rep was ...,"[charg, overwhelm, comcast, servic, rep, ignor..."
9,"I have had cable, DISH, and U-verse, etc. in t...","[cabl, dish, uvers, etc, past, eh, know, comca..."


In [25]:
#Train Word2Vec model
tokenized_dataset = [doc.split() for doc in df]
word2vec_model = Word2Vec(sentences=tokenized_dataset, vector_size=100, window=5, min_count=1, workers=4)

In [27]:
#Create document embedding
X = np.array([np.mean([word2vec_model.wv[word] for word in doc.split() if word in word2vec_model.wv], axis=0) for doc in df])

In [28]:
#Perform clustering
k = 2  # Define the number of clusters
km = KMeans(n_clusters=k)
km.fit(X)

C:\Users\cyber\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\cyber\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=2)

In [29]:
#Predict the clusters for each document
y_pred = km.predict(X)

In [31]:
#Tabulate the document and predicted cluster
table_data = [["Document", "Predicted Cluster"]]
table_data.extend([[doc, cluster] for doc, cluster in zip(df, y_pred)])
print(tabulate(table_data, headers="firstrow"))

Document                Predicted Cluster
--------------------  -------------------
Author                                  0
posted_on                               0
rating                                  1
Text                                    1
Clean_Punctuation                       1
Clean_Lower                             1
Clean_Number                            1
Tokenize_Data                           1
Clean_Stopwords                         1
Clean_StemmedWord                       1
Clean_LemmatizedWord                    0


In [32]:
#Calculate purity
total_samples = len(y_pred)
cluster_label_counts = [Counter(y_pred)]
purity = sum(max(cluster.values()) for cluster in cluster_label_counts) / total_samples

print("Purity:", purity)

Purity: 0.7272727272727273
